<img src="img/encabezado_EDA.png" alt="encabezado" width="1000" height="600"/>

# PASO 1: importación de librerías necesarias

In [1]:
import ast
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# PASO 2: limpieza y tratamiento de datos

In [2]:
# Convertimos la base de datos a dataframe
df_goodreads = pd.read_csv("./data/goodreads_data.csv")

In [3]:
# Vemos la información del dataframe
df_goodreads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


De esta primera lectura observamos que la columna "Description", que es la que presenta nulos, podemos borrarla ya que no aporta ninguna información necesaria a futuro.
Del mismo modo podemos eliminar la columna URL.

In [4]:
df_goodreads = df_goodreads.drop(columns=['Description','URL'])
df_goodreads.head()

,Unnamed: 0,Book,Author,Genres,Avg_Rating,Num_Ratings
0,0,To Kill a Mockingbird,Harper Lee,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311"
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135"
2,2,Pride and Prejudice,Jane Austen,"['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155"
3,3,The Diary of a Young Girl,Anne Frank,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438"
4,4,Animal Farm,George Orwell,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172"


En cuanto a la columna Num_Ratings debería ser de tipo int y no object, por lo que debemos realizar el cambio.

In [5]:
df_goodreads['Num_Ratings']

0       5,691,311
1       9,278,135
2       3,944,155
3       3,488,438
4       3,575,172
          ...    
9995          276
9996           60
9997          383
9998          263
9999           36
Name: Num_Ratings, Length: 10000, dtype: object

In [6]:
# Quitamos las comas y las sustituimos por nada:
df_goodreads['Num_Ratings'] = df_goodreads['Num_Ratings'].str.replace(',', '')

# Cambiamos el tipo a int.
df_goodreads['Num_Ratings'] = df_goodreads['Num_Ratings'].astype(int)

In [7]:
df_goodreads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Genres       10000 non-null  object 
 4   Avg_Rating   10000 non-null  float64
 5   Num_Ratings  10000 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 468.9+ KB


Por otro lado, existe la posibilidad de que haya libros duplicados, por lo tanto debemos comprobarlo y así tratarlos.

In [8]:
duplicados = df_goodreads[df_goodreads.duplicated(subset = ['Book', 'Author'], keep = False)]
duplicados

,Unnamed: 0,Book,Author,Genres,Avg_Rating,Num_Ratings
21,21,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963002
59,59,The Color Purple,Alice Walker,"['Classics', 'Fiction', 'Historical Fiction', ...",4.26,633920
125,125,The Last Lecture,Randy Pausch,"['Nonfiction', 'Memoir', 'Biography', 'Self He...",4.26,334149
134,134,"The Girl with the Dragon Tattoo (Millennium, #1)",Stieg Larsson,"['Fiction', 'Mystery', 'Thriller', 'Crime', 'M...",4.16,3029842
231,231,Speak,Laurie Halse Anderson,"['Young Adult', 'Fiction', 'Contemporary', 'Re...",4.04,565317
...,...,...,...,...,...,...
8467,8467,One Plus One,Jojo Moyes,"['Fiction', 'Romance', 'Chick Lit', 'Contempor...",3.96,166104
8608,8608,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186
8626,8626,Shakespeare's Sonnets,William Shakespeare,"['Poetry', 'Classics', 'Fiction', 'Romance', '...",4.22,2493
8678,8678,"The Oldest Dance (Wisdom Revolution, #2)",Misba,"['Fantasy', 'Science Fiction Fantasy', 'Post A...",4.48,5547


Existen 198 filas duplicadas como por ejemplo las que se muestran a continuación. 

In [21]:
libro_duplicado = df_goodreads[df_goodreads['Book'] == 'The Hunger Games (The Hunger Games, #1)']
libro_duplicado

,Unnamed: 0,Book,Author,Genres,Avg_Rating,Num_Ratings
21,21,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963002
4868,4868,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186
4894,4894,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186
8608,8608,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186


In [20]:
libro_duplicado = df_goodreads[df_goodreads['Book'] == 'The Color Purple']
libro_duplicado

,Unnamed: 0,Book,Author,Genres,Avg_Rating,Num_Ratings
59,59,The Color Purple,Alice Walker,"['Classics', 'Fiction', 'Historical Fiction', ...",4.26,633920
272,272,The Color Purple,Alice Walker,"['Classics', 'Fiction', 'Historical Fiction', ...",4.26,633926


Como podemos observar, lo más acertado sería quedarse con el último valor registrado que es el que presenta cambios.

Para no perder las lecturas de los libros:

In [ ]:
df_suma_lecturas = df_goodreads.groupby('Book')['Num_Ratings'].sum().reset_index()
df_suma_lecturas

In [ ]:
libro_duplicado = df_suma_lecturas[df_suma_lecturas['Book'] == 'Exodus']
libro_duplicado

In [ ]:
df_merged = pd.merge(df_goodreads, df_suma_lecturas, on='Book', how='left', suffixes=('', '_sum'))
df_merged

In [ ]:
df_merged['Num_Ratings'] = df_merged['Num_Ratings_sum']
df_merged.drop(columns=['Num_Ratings_sum'], inplace=True)

In [ ]:
df_merged = df_merged.drop_duplicates(subset='Book', keep='first')

In [ ]:
libro_duplicado = df_merged[df_merged['Book'] == 'Exodus']
libro_duplicado

In [ ]:
df_goodreads = df_merged

La columna "Genres" está formada por una lista de distintos tipos de género.
Vamos a suponer que el género principal es el primero que aparece y lo indicaremos en una nueva columna.

In [ ]:
def obtener_primer_valor(x):
    try:
        lista = ast.literal_eval(x)  # Convertir la cadena a lista
        if len(lista) > 0:  # Verificar si la lista no está vacía
            return lista[0]  # Devolver el primer valor
        else:
            return None  # Si la lista está vacía, devolver None (o cualquier valor por defecto)
    except (ValueError, SyntaxError):  # Capturar errores de conversión de cadenas no válidas
        return None  # Si la cadena no es válida, devolver None

# Aplicar la función a la columna
df_goodreads['Principal_Genre'] = df_goodreads['Genres'].apply(obtener_primer_valor)


In [ ]:
df_goodreads.head(10)

In [ ]:
# Observamos cuáles son los valores de la nueva columna
df_goodreads['Principal_Genre'].value_counts()

Existe géneros que seguramente se puedan agrupar en otros, por ejemplo Military History o Civil War podrían ser libros de género Histórico o Fae podría ser fantasía.

In [ ]:
# Observamos que libros son:
generos_filtrados = 'Fae', 'Womens', 'Military History', 'Wicca', 'Civil War'
df_goodreads[df_goodreads['Principal_Genre'].isin(generos_filtrados)]

In [ ]:
df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace({'Civil War': 'History', 'Military History': 'History'})

In [ ]:
df_goodreads['Principal_Genre'].value_counts()

Como sigue habiendo muchos géneros con un solo valor vamos a analizar cuáles son para poder limpiarlo lo máximo posible.

In [ ]:
# Obtener los valores con un recuento de 1
generos = df_goodreads['Principal_Genre'].value_counts()
generos_unicos = generos[generos == 1]

# Ver los valores con recuento 1
generos_unicos

In [ ]:
# Escogemos algunos ejemplos para ver si presentan otros géneros a los que poder asociarlos.
generos_filtrados = 'Survival', 'Social Justice', 'Personal Development', 'High School', 'Magical Realism', 'Canada', 'Cooking', 'Horses'
df_goodreads[df_goodreads['Principal_Genre'].isin(generos_filtrados)]

Para poder limpiar los datos usaremos dos técnicas.
* La primera consiste en reemplazar los valores actuales por los nuevos como si se tratara de un diccionario. Esto aplica a un género que es cambido por otro.
* La segunda técnica consiste en cambiar varios géneros a la vez.

In [ ]:
reemplazar = {'Social Justice': 'History','Survival': 'Adventure', 'Personal Development': 'Self Help',
              'High School': 'Romance', 'Canada': 'Historical Fiction', 'Comedy' : 'Humor','Judaism' : 'Religion',
               'Horses': 'Animals', 'Linguistics' : 'Linguistics', 'Transgender' : 'Memoir', 'Linguistics' : 'Philosophy',
             'Juvenile': 'Young Adult', 'Kids' : 'Childrens', 'Gaming' : 'Business', 'Suspense': 'Thriller', 'Light Novel' : 'Manga',
             'Architecture': 'Design', 'Shapeshifters' : 'Paranormal', 'comunication': 'Psychology', 'Gender': 'Feminism',
             'Pulp': 'Amazon', 'Marathi':'Mythology', 'Adult': 'New Adult', 'Anthropology': 'Essays', 'Ukrainian Literature': 'Classics',
             'Literary Criticism': 'Literature', 'Medicine': 'Medical', 'Portugal': 'Short Stories', 'Geology': 'Science' }

df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace(reemplazar)

In [ ]:
df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace(
    to_replace=r'.*(Love|Romance|Teen).*',  # Expresión regular para encontrar "love","romance", etc
    value='Romance',                   # Nuevo valor
    regex=True                         # Usar regex
)


df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace(
    to_replace=r'.*(Literary Fiction|Cyberpunk|Speculative Fiction|Womens Fiction|Russia|German Literature|African American|Indian Literature|Indian Literature ).*',  # Expresión regular para encontrar "love" o "romance"
    value='Fiction',                   # Nuevo valor
    regex=True                         # Usar regex
)


df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace(
    to_replace=r'.*(Magical Realism|Gods|Angels|Fae|Fairy Tales|Epic Fantasy|Lds).*',  
    value='Fantasy',                   # Nuevo valor
    regex=True                         # Usar regex
)

df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace(
    to_replace=r'.*(Cooking|Vegetarian).*', 
    value='Food',                   # Nuevo valor
    regex=True                         # Usar regex
)

df_goodreads['Principal_Genre'] = df_goodreads['Principal_Genre'].replace(
    to_replace=r'.*(Christian Non Fiction|Non Fiction|Race|Death|Film|Law|Martial Arts|Prayer).*', 
    value='Nonfiction',                   # Nuevo valor
    regex=True                         # Usar regex
)

In [ ]:
generos = df_goodreads['Principal_Genre'].value_counts()
generos_unicos = generos[generos == 1]

# Ver los valores con recuento 1
generos_unicos.index

Ahora solo tenemos 15 géneros con un único valor. Estos datos no podemos limpiarlos más ya que sólo tienen un género asignado.

# Paso 3: Hipótesis

Antes de comenzar con el análisis debemos tener muy claras las hipótesis y cuáles son nuestros objetivos a la hora de realizar el análisis.
Con este EDA queremos analizar y dar respuesta a las siguientes preguntas:
1. ¿Cuál es el género literario más leído?
2. ¿ Es el género literario más leído el mejor puntuado?
3. ¿ Existe alguna relación entre los autores más publicados y los más leídos?
4. ¿Qué autores destacan en los géneros más leídos?
5. ¿Qué autores destacan en el género menos popular? 

# Paso 4: Análisis

> ## Relación entre géneros literarios más leídos y sus puntuaciones.

In [ ]:
# Obtenemos con cuántos géneros literarios nos hemos quedado:
genero_unicos = df_goodreads['Principal_Genre'].nunique()
genero_unicos

In [ ]:
# Como siguen siendo muchos me interesa saber cuáles son los 10 géneros literarios más comunes.
generos_mas_comunes = df_goodreads['Principal_Genre'].value_counts().head(10)
generos_mas_comunes

In [ ]:
cantidad_lecturas = df_goodreads.groupby('Principal_Genre')['Num_Ratings'].sum().sort_values(ascending=False).head(10)
cantidad_lecturas

### La primera observación que tenemos es que los géneros más leídos coinciden en su mayoría con los géneros más publicados.

A continuación vamos a observar qué sucede con las puntuaciones según los géneros principales. Para ello debemos crear una columna que tenga en cuenta tanto la puntuación como el total de lecturas.

In [ ]:
media_valoraciones = df_goodreads.groupby('Principal_Genre')['Avg_Rating'].mean().head(10).sort_values(ascending = False)
media_valoraciones

Si simplemente vemos cuáles son los géneros de mayor puntuación se verán favorecidos aquellos géneros que se leen menos.

Creamos un dataframe con los géneros más leídos y calculamos la puntuación media de dichos géneros.

In [ ]:
df_filtrado = df_goodreads[df_goodreads['Principal_Genre'].isin(cantidad_lecturas.index)]
df_filtrado

In [ ]:
media_valoraciones = df_filtrado.groupby('Principal_Genre')['Avg_Rating'].mean().sort_values(ascending=False)
media_valoraciones

### Queremos saber si existe una correlacion positiva entre géneros más leidos y sus puntuaciones

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtrado, x='Num_Ratings', y='Avg_Rating', hue='Principal_Genre', palette='Set2', s=100)

A simple vista tenemos demasiados datos para saber si existe alguna correlación, aunque aparentemente los valores se distribuyen entorno a una línea recta lo que significaría que no hay correlación.

In [ ]:
# 1. Creamos un gráfico que una los generos más leidos y la media de valoraciones.
df_agg = pd.DataFrame({
    'Avg_Rating': media_valoraciones,
    'Num_Ratings': cantidad_lecturas})

df_agg

In [ ]:
# 2. Creamos el diagrama de dispersión.
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_agg, x='Num_Ratings', y='Avg_Rating', hue=df_agg.index, palette='Set2', s=100)
plt.xlabel('Cantidad de Lecturas', fontsize=12)
plt.ylabel('Puntuación Media', fontsize=12);

In [ ]:
df_agg = df_agg.reset_index()
df_agg.sort_values('Num_Ratings', ascending=False).plot(kind='bar', x='Principal_Genre', y='Avg_Rating', legend=False, color='brown')
plt.title('Comparación de Puntuación Media según Géneros Más Leídos')
plt.xlabel('Género')
plt.ylabel('Puntuación Media')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Sin embargo, a pesar de lo mencionado con anterioridad, observamos que géneros como 'Horror' son menos leídos y también peor puntuados al contrario que 'Fantasy' que es el género más leído y también el mejor puntuado

Además encontramos dos claros grupos, los que son leidos por mas de 100 millones de personas y los que no,
En el primer grupo destaca el género 'Horror' como el peor puntuado y el menos leído. Por otro lado, 'Historical Fiction' y Nonfiction' son los más leídos y los mejores puntuados.

En el segundo grupo destaca 'Fantasy como el género más leído y mejor puntuado'.
En este grupo encontramos que excepto el género anterior se puntuan mucho peor, tal y como sucede con'Fiction' que es el más leído de todos y el peor puntuado.

Podemos mirar relaciones entre autores y mejores puntuaciones o que autores tienen mas publicaciones

> ## Relación entre autores literarios más leídos y más publicados

In [ ]:
# Antes de comenzar veamos cuántos autores tenemos en la base de datos.
autores = df_goodreads['Author'].nunique()
autores

In [ ]:
# 1. Obtenemos cuáles son los autores que más libros publicados tienen.
autores_publicados = df_goodreads['Author'].value_counts()
autores_publicados

In [ ]:
mediana_autores = autores_publicados.median()
mediana_autores

In [ ]:
# 2. Obtenemos cuáles son los autores que más leídos.
autores_mas_leidos = df_goodreads.groupby('Author')['Num_Ratings'].sum().sort_values(ascending = False)
autores_mas_leidos

In [ ]:
# 3. Unimos ambos datafarmes para obtener la correlacion.
df_autores = pd.DataFrame({
    'publicaciones': autores_publicados,
    'lecturas': autores_mas_leidos
})

# 4. Obtenemos la correlacion
correlacion = round(df_autores['publicaciones'].corr(df_autores['lecturas']),2)
correlacion

La correlacion obtenida es aproximadamente 0,46. Esto significa que existe una correlacion positiva moderada. Es decir, podríamos determinar que cuanto más publiquen a un autor, más libros se van a leer. Sin embargo, al ser moderada no quiere decir que esto sea determinante, sino que existirán otras causas que hagan que los libros sean más leídos.

> ## Relación entre autores y sus géneros literarios.

In [ ]:
conteo_generos = df_goodreads.groupby(['Author', 'Principal_Genre']).size().reset_index(name='Cantidad').sort_values(by = ['Cantidad', 'Author', 'Principal_Genre'],ascending = [False, True, True])
conteo_generos

Para ver la correlación entre autor, género y cantidad de lecturas debemos acudir a hacer un ANOVA de dos vías.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# ANOVA de dos vías para autor y genero
modelo = ols('Num_Ratings ~ C(Author) + C(Principal_Genre) + C(Author):C(Principal_Genre)', data=df_goodreads).fit()
anova_resultados = anova_lm(modelo)

print(anova_resultados)


In [ ]:
ANOVA de dos vías: Permite analizar la interacción entre autor y género para ver si ambos factores juntos afectan las lecturas de forma significativa.

El modelo lecturas ~ C(autor) + C(genero) + C(autor):C(genero) incluye tanto los efectos principales de autor y género como la interacción entre ellos.
En los resultados, si la columna PR(>F) para C(autor):C(genero) es significativa (p < 0.05), significa que hay una interacción significativa entre el autor y el género respecto a las lecturas.
3. Interpretación de los resultados:
Si el valor p es menor que 0.05, puedes concluir que existe una relación significativa entre las lecturas y la variable que estás analizando (autor o género).
Si el valor p es mayor que 0.05, no hay suficiente evidencia para concluir que exista una relación significativa.

Me sale ese error porque no es capaz de manejar tantos tipos de valores.

> # Autores más leídos en los géneros más populares

Tal y como se observó con anterioridad los géneros que más lectores presentan son 'Fantasy' y 'Fiction', pero observamos que cada uno de ellos tenía puntuaciones muy diferentes.

Es por ello, que procederemos a un análisis individual de cada tipo para ver sus autores y las puntuaciones.

>> ## Autores más leídos en 'Fantasy'

In [ ]:
# 1. Obtenemos un dataframe que contenga todos los libros del género fantasía.
df_fantasy = df_goodreads[df_goodreads['Principal_Genre'] == 'Fantasy']

# 2. Lo ordenamos para obtener cuáles son los 5 libros más leídos.
df_fantasy_top5 = df_fantasy.sort_values(by= 'Num_Ratings', ascending = False).head(5)

df_fantasy_top5

In [ ]:
# 3. Creamos el gráfico de barras.
plt.figure(figsize=(10, 6))
plt.bar(df_fantasy_top5['Book'], df_fantasy_top5['Num_Ratings'], color='brown')

# Títulos y etiquetas
plt.title('Top 5 Libros más leídos', fontsize=14)
plt.xlabel('Libro', fontsize=12)
plt.ylabel('Lecturas', fontsize=12)

#  Rotar los títulos de los libros para mejor visibilidad
plt.xticks(rotation=45, ha='right');

Tras un primer análisis observamos que 3 de los 5 libros de Fantasía más populares pertenecen a la autora J.K. Rowling.
Pero, ¿será también la autora más leída en general? ¿Y la mejor puntuada?

Comencemos con el primero de estos análisis.

In [ ]:
autores_fantasy = df_fantasy.groupby('Author')['Num_Ratings'].sum().sort_values(ascending = False).head (5)
autores_fantasy

In [ ]:
# Gráfico
plt.figure(figsize=(10, 6))
plt.bar(autores_fantasy.index, autores_fantasy.values, color='brown')

# Títulos y etiquetas
plt.title('Top 5 Autores más leídos', fontsize=14)
plt.xlabel('Autores', fontsize=12)
plt.ylabel('Lecturas', fontsize=12);



En este caso han aparecido autores como Rick Riordan y Sarah J.Mass entre los más leídos. Sin embargo, J.K. Rowling muestra una gran ventaja respecto a los demás.

Ahora bien, puede que estos autores sean los más leídos pero, ¿son los mejor puntuados?

In [ ]:
autores_fantasy_puntuacion = df_fantasy.groupby('Author')['Avg_Rating'].mean().sort_values(ascending = False).head (5)
autores_fantasy_puntuacion

Como vemos, ninguno de los autores más leídos está dentro de los 5 mejor puntuados. Esto se debe a que cuanta más gente te lee y más libros publicados tienes, la puntuación se ve más distribuida que para otros autores que solo tengan un libro publicado y con buena puntuación.

Pero veamos que puntuaciones medias tienen los autores más leídos.

In [ ]:
autores_seleccionados = ['J.K. Rowling', 'Rick Riordan', 'Sarah J. Maas', 'Stephenie Meyer', 'J.R.R. Tolkien']
df_autores = df_fantasy[df_fantasy['Author'].isin(autores_seleccionados)]
autores_fantasy_puntuacion = df_autores.groupby('Author')['Avg_Rating'].mean().sort_values(ascending=False)
autores_fantasy_puntuacion

In [ ]:
plt.figure(figsize=(10, 6))
colores = {'J.K. Rowling': 'orange', 'Rick Riordan': 'red','Sarah J. Maas': 'blue','Stephenie Meyer': 'black', 'J.R.R. Tolkien' : 'green'}

for autor, color in colores.items():
    df_autor = df_autores[df_autores['Author'] == autor]
    plt.scatter(df_autor['Num_Ratings'], df_autor['Avg_Rating'], color=color, s=100, label=autor)

plt.title('Distribución de Lectores y Puntuaciones Promedio de Autores', fontsize=14)
plt.xlabel('Número de Lectores', fontsize=12)
plt.ylabel('Puntuación Promedio', fontsize=12)

plt.legend(title="Autores")

Podemos obtener las siguientes conclusiones:
1. J.K.Rowling se sitúa entre las mejor puntuadas, seguida por Sarah J.Mass.
2. Rick Riordan es el que más libros ha publicado superando todos ellos la puntuación de 4.
3. Stephanie Meyer tiene libros muy populares pero no muy bien valorados.
4. J.R.R. Tolkien ha escrito libros peor puntuados que los de Sarah J.Mass pero más populares.

>> ## Autores más leídos en 'Fiction'

In [ ]:
# 1. Obtenemos un dataframe que contenga todos los libros del género fantasía.
df_fiction = df_goodreads[df_goodreads['Principal_Genre'] == 'Fiction']

# 2. Lo ordenamos para obtener cuáles son los 5 libros más leídos.
df_fiction_top5 = df_fiction.sort_values(by= 'Num_Ratings', ascending = False).head(5)

df_fiction_top5

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(df_fiction_top5['Book'], df_fiction_top5['Num_Ratings'], color='brown')

# Títulos y etiquetas
plt.title('Top 5 Libros más leídos', fontsize=14)
plt.xlabel('Libro', fontsize=12)
plt.ylabel('Lecturas', fontsize=12)

#  Rotar los títulos de los libros para mejor visibilidad
plt.xticks(rotation=45, ha='right');

In [ ]:
autores_fiction = df_fiction.groupby('Author')['Num_Ratings'].sum().sort_values(ascending = False).head (5)
autores_fiction

In [ ]:
autores_seleccionados_fiction = ['Dan Brown', 'Stieg Larsson', 'Taylor Jenkins Reid', 'Khaled Hosseini', 'John Grisham']
df_autores_fiction = df_fiction[df_fiction['Author'].isin(autores_seleccionados_fiction)]
autores_fiction_puntuacion = df_autores_fiction.groupby('Author')['Avg_Rating'].mean().sort_values(ascending=False)
autores_fiction_puntuacion

In [ ]:
plt.figure(figsize=(10, 6))
colores = {'Dan Brown': 'orange', 'Stieg Larsson': 'red','Taylor Jenkins Reid': 'blue','Khaled Hosseini': 'black', 'John Grisham' : 'green'}

for autor, color in colores.items():
    df_autor_fiction = df_autores_fiction[df_autores_fiction['Author'] == autor]
    plt.scatter(df_autor_fiction['Num_Ratings'], df_autor_fiction['Avg_Rating'], color=color, s=100, label=autor)

plt.title('Distribución de Lectores y Puntuaciones Promedio de Autores de Ficción', fontsize=14)
plt.xlabel('Número de Lectores', fontsize=12)
plt.ylabel('Puntuación Promedio', fontsize=12)

plt.legend(title="Autores de Ficción")

### Conclusiones
1. Lo primero que se observa es que el eje y, donde se representan las puntuaciones ha disminuido de valores.
2. John Grisham ha mantenido su público pero ha ido mejorando sus puntuaciones.
3. Dan Brown ha ido creciendo en lectores pero no consigue llegar con ninguno de sus libros a 4.
4. Stieg Larsson, Taylor Jenkins Reid y Khaled Hosseini tienen una distribución similar teniendo pocos libros publicados pero con gran público que les da unas buenas valoraciones.

In [ ]:
plt.figure(figsize=(10, 6))

# Graficar los puntos: eje X = 'Num_Ratings' (lecturas), eje Y = 'Avg_Rating' (puntuación promedio)
plt.scatter(df_fantasy['Num_Ratings'], df_fantasy['Avg_Rating'], color='brown', alpha=0.5)

# Título y etiquetas
plt.title('Distribución de Lectores y Puntuaciones Promedio para el Género de Fantasía', fontsize=14)
plt.xlabel('Número de Lectores', fontsize=12)
plt.ylabel('Puntuación Promedio', fontsize=12)

In [ ]:
plt.figure(figsize=(10, 6))

# Graficar los puntos: eje X = 'Num_Ratings' (lecturas), eje Y = 'Avg_Rating' (puntuación promedio)
plt.scatter(df_fiction['Num_Ratings'], df_fiction['Avg_Rating'], color='brown', alpha=0.5)

# Título y etiquetas
plt.title('Distribución de Lectores y Puntuaciones Promedio para el Género de Ficción', fontsize=14)
plt.xlabel('Número de Lectores', fontsize=12)
plt.ylabel('Puntuación Promedio', fontsize=12)

Hay más valores entre 1 M y 3 M en Ficción.
Las puntuaciones de Fantasy se cpncentrán entre 3,5 y 4,8 mientras que las de Ficción entre 3 y 4,6

> # Género Horror

In [ ]:
# 1. Obtenemos un dataframe que contenga todos los libros del género fantasía.
df_horror = df_goodreads[df_goodreads['Principal_Genre'] == 'Horror']

# 2. Lo ordenamos para obtener cuáles son los 5 libros más leídos.
df_horror_top5 = df_horror.sort_values(by= 'Num_Ratings', ascending = False).head(5)

df_horror_top5

In [ ]:
plt.figure(figsize=(10, 6))

# Graficar los puntos: eje X = 'Num_Ratings' (lecturas), eje Y = 'Avg_Rating' (puntuación promedio)
plt.scatter(df_horror['Num_Ratings'], df_horror['Avg_Rating'], color='brown', alpha=0.5)

# Título y etiquetas
plt.title('Distribución de Lectores y Puntuaciones Promedio para el Género de Horror', fontsize=14)
plt.xlabel('Número de Lectores', fontsize=12)
plt.ylabel('Puntuación Promedio', fontsize=12)

Este gráfico muestra la relación entre el número de lectores y la puntuación promedio para libros del género horror. Aquí tienes algunas conclusiones relevantes:

Alta concentración de libros con pocos lectores: La mayoría de los libros tienen menos de 200,000 lectores, como se observa en la concentración de puntos en el extremo izquierdo del eje X.

Puntuaciones promedio entre 3.5 y 4.5: Aunque hay variaciones, la mayoría de los libros obtienen puntuaciones promedio que oscilan entre 3.5 y 4.5, lo que indica que los lectores califican estos libros generalmente de manera favorable.

Libros con más lectores tienden a tener puntuaciones más altas:

Los libros con más de 600,000 lectores tienden a tener puntuaciones promedio alrededor de 4.0 o más.
Esto sugiere que los libros populares en el género de horror suelen ser mejor valorados.
Variabilidad en libros menos populares: Para libros con menos lectores (menos de 200,000), las puntuaciones son más dispersas. Esto podría significar que:

Algunos libros son valorados muy positivamente aunque tengan pocos lectores.
Otros reciben puntuaciones más bajas.
Libros extremadamente populares son pocos:

Solo unos pocos libros tienen más de 1,000,000 de lectores.
Estos libros tienden a destacarse con puntuaciones promedio superiores a 4.0.
En resumen, la popularidad de los libros del género horror (medida por el número de lectores) parece estar correlacionada con puntuaciones más altas, pero existe una gran cantidad de libros con menor alcance y con puntuaciones variadas. Esto podría indicar un mercado con una base sólida de obras populares, pero también una amplia variedad de libros menos conocidos.